In [1]:
import os
import json

# directory where JSON files are located
directory = os.getcwd()

# get all files in the current working directory
files = os.listdir(directory)

# filter out only JSON files
json_files = [file for file in files if file.endswith(".json")]

# initialize a dictionary to store the data from all the JSON files
data = {}

# helper function to merge two dictionaries with nested lists


def merge_dicts(dict1, dict2):
    for key, value in dict2.items():
        if key in dict1 and isinstance(value, (list, dict)):
            # if the key exists in both dictionaries and the value is a list or dictionary, recursively merge the nested data
            if isinstance(value, list):
                # if the value is a list, merge the lists
                dict1[key] = dict1[key] + value
            else:
                # if the value is a dictionary, recursively merge the dictionaries
                dict1[key] = merge_dicts(dict1[key], value)
        else:
            # otherwise, just add the key-value pair from dict2 to dict1
            dict1[key] = value
    return dict1


# load and parse each JSON file, and merge the data into the dictionary
for file in json_files:
    with open(os.path.join(directory, file)) as json_file:
        file_data = json.load(json_file)
        data = merge_dicts(data, file_data)

if not os.path.exists("output"):
    os.makedirs("output")

# save the merged data to a new JSON file
with open("output/merged_data.json", "w") as outfile:
    json.dump(data, outfile)

data = json.load(open(os.path.join(directory, "output/merged_data.json")))
    



In [2]:
def score_path(path,trails,tree):
    score = 0
    for trail in path:
        score +=trails[trail]["distance"]
    return score
    

In [3]:
def decode(encoded):
    path = []
    n = [0, 0]
    j = 0
    for i in range(len(encoded)):
        k = ord(encoded[i]) - 0x30
        n[j] = (n[j] << 5) | (k & 0x1f)
        if k & 0x20:
            n[j] = (~(n[j] >> 1) if n[j] & 0x01 else n[j] >> 1) * 1e-7
            if j == 1:
                latLon = path[-1] if len(path) > 0 else (0, 0)
                path.append((latLon[0] + n[0], latLon[1] + n[1]))
            j = (j + 1) % 2
            n[j] = 0
    return (path[0], path[-1])

In [4]:
for trail in data["trails"].values():
    # print(trail)
    startPos, endPos = decode(trail.pop("encoded"))
    trail["start_latlng"] = (round(startPos[0], 6), round(startPos[1], 6))
    trail["end_latlng"] = (round(endPos[0], 6), round(endPos[1], 6))


In [5]:
import numpy as np
print(len(data["nodes"]))
data["nodes"] = {key: value for (key, value) in data["nodes"].items() if value["region"] == "Tatry"}
print(len(data["nodes"]))

start_points = ["Siwa Polana", "Polana Biały Potok", "Kiry", "Staników Żleb", "Nędzówka, las", "Gronik", "Za Bramką", "Dolina Strążyska, parking",
                "Dolina ku Dziurze", "Dolina Białego", "Kuźnicka Polana", "Kuźnice", "Jaszczurówka", "Lysá Poľana, parking", "Palenica Białczańska", "Tatranská Javorina", "Monkova dolina", "Tatranská Kotlina", "Nad Matliarmi", "Hrebienok", "Velický most", "Nad Zrubami", "Trigan", "Pod Furkotskou doliou", "Pod Grúnikom", "Podbanské, rozc.", "Úzka dolina, parking", "Pod Holým vrchom", "Žiarska dolina, ústie", "Jalovecká dolina, ústie", "Biela skala, horáreň", "Brestová, skanzen", "Rázcestie pod Suchou dolinou"]

start_points_ids = []

for i,node in enumerate(data["nodes"].values()):
    node["id"]=i
    if node["name"] in start_points:
        start_points_ids.append(i)

print(start_points_ids)


matrix = np.full((len(data["nodes"]), len(data["nodes"])),np.inf)
for node in data["nodes"].values():
    for trail in data["trails"].values():
        if trail["start_latlng"] == (node["latitude"], node["longitude"]):
            for node2 in data["nodes"].values():
                if trail["end_latlng"] == (node2["latitude"], node2["longitude"]):
                    matrix[node["id"], node2["id"]] = trail["start_end_time"]
                    matrix[node2["id"], node["id"]] = trail["end_start_time"]
        if trail["end_latlng"] == (node["latitude"], node["longitude"]):
            for node2 in data["nodes"].values():
                if trail["start_latlng"] == (node2["latitude"], node2["longitude"]):
                    matrix[node["id"], node2["id"]] = trail["start_end_time"]
                    matrix[node2["id"], node["id"]] = trail["end_start_time"]

# print(matrix)
np.set_printoptions(threshold=np.inf, linewidth=np.inf)
with open("matrix.txt", 'w') as f:
    f.write(np.array2string(matrix, separator=', '))
# matrix.tofile("matrix.txt", sep=', ', format='%s')

        

764
336
[4, 5, 11, 23, 47, 50, 72, 76, 81, 92, 102, 115, 124, 144, 150, 162, 173, 175, 177, 188, 200, 207, 245, 247, 286, 312, 317, 319, 329, 332, 333]


In [6]:
start_points = ["Siwa Polana", "Polana Biały Potok", "Kiry", "Staników Żleb", "Nędzówka, las", "Gronik", "Za Bramką", "Dolina Strążyska, parking",
                "Dolina ku Dziurze", "Dolina Białego", "Kuźnicka Polana", "Kuźnice", "Jaszczurówka", "Lysá Poľana, parking", "Palenica Białczańska", "Tatranská Javorina", "Monkova dolina", "Tatranská Kotlina", ""]

for point in start_points:
    point=point.encode("UTF-16")

In [7]:
print(np.count_nonzero(matrix))
print(len(data["trails"].values()))


112896
971


In [8]:
nodes = data["nodes"].values()
